In [15]:
import pickle
import re
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [16]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [17]:
import warnings
warnings.filterwarnings('ignore')

In [18]:
def download_nltk_resources():
    try:
        nltk.download('punkt', quiet=True)
        nltk.download('stopwords', quiet=True)
        # Additional Punkt resources needed
        nltk.download('punkt_tab', quiet=True)
        nltk.download('wordnet', quiet=True)
        nltk.download('omw-1.4', quiet=True)
    except Exception as e:
        print(f"Error downloading NLTK resources: {e}")
        print("Attempting to continue with available resources...")
download_nltk_resources()

In [19]:
# Verify Punkt tokenizer is available
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    print("Punkt tokenizer not found - some text processing may fail")
    try:
        nltk.download('punkt')
    except:
        print("Could not download Punkt tokenizer")

In [20]:
# Load dataset
df = pd.read_csv('/content/drive/MyDrive/Symptom2Disease.csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [21]:
# Improved preprocessing function
def preprocess_text(text):
    # Remove special chars, keep commas for symptom separation
    text = re.sub(r'[^a-zA-Z,\s]', '', text)
    # Replace commas with spaces (for inputs like "headache,fever")
    text = text.replace(',', ' ')
    # Tokenize and stem
    tokens = word_tokenize(text.lower())
    stemmer = SnowballStemmer('english')
    tokens = [stemmer.stem(token) for token in tokens if token.isalpha()]
    return ' '.join(tokens)

In [22]:
# Apply preprocessing
df['processed_text'] = df['text'].apply(preprocess_text)

# TF-IDF with unigrams and bigrams
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), stop_words='english')
X = tfidf.fit_transform(df['processed_text'])

# Label encoding
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])

In [23]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Model ensemble
models = [
    ('nb', MultinomialNB(alpha=0.1)),
    ('rf', RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)),
    ('lr', LogisticRegression(max_iter=1000, C=1, solver='liblinear')),
    ('svm', SVC(kernel='linear', probability=True, C=0.5, random_state=42))
]
voting_clf = VotingClassifier(estimators=models, voting='soft')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('nb', MultinomialNB(alpha=0.1)),
                             ('rf',
                              RandomForestClassifier(max_depth=10,
                                                     n_estimators=200,
                                                     random_state=42)),
                             ('lr',
                              LogisticRegression(C=1, max_iter=1000,
                                                 solver='liblinear')),
                             ('svm',
                              SVC(C=0.5, kernel='linear', probability=True,
                                  random_state=42))],
                 voting='soft')

In [24]:
# Evaluation
y_pred = voting_clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Accuracy: 0.975

Classification Report:
                                  precision    recall  f1-score   support

                           Acne       1.00      1.00      1.00        10
                      Arthritis       1.00      1.00      1.00        10
               Bronchial Asthma       1.00      1.00      1.00        10
           Cervical spondylosis       1.00      1.00      1.00        10
                    Chicken pox       1.00      0.80      0.89        10
                    Common Cold       1.00      1.00      1.00        10
                         Dengue       1.00      1.00      1.00        10
          Dimorphic Hemorrhoids       1.00      1.00      1.00        10
               Fungal infection       1.00      1.00      1.00        10
                   Hypertension       1.00      1.00      1.00        10
                       Impetigo       1.00      1.00      1.00        10
                       Jaundice       1.00      1.00      1.00        10
         

In [25]:
# Save the complete model pipeline
with open('symptom_classifier_ensemble.pkl', 'wb') as f:
    pickle.dump({
        'model': voting_clf,
        'vectorizer': tfidf,
        'label_encoder': label_encoder,
        'preprocess_text': preprocess_text,
        'stemmer': SnowballStemmer('english')
    }, f)
    print("Ensemble model saved as symptom_classifier_ensemble.pkl")

Ensemble model saved as symptom_classifier_ensemble.pkl


In [26]:
# Example of how to load and use the saved model
def load_and_predict(model_path, symptoms, top_n=3):
    with open(model_path, 'rb') as f:
        model_data = pickle.load(f)

    processed = model_data['preprocess_text'](symptoms)
    vector = model_data['vectorizer'].transform([processed])
    probs = model_data['model'].predict_proba(vector)[0]

    classes = model_data['label_encoder'].classes_
    top_preds = sorted(zip(classes, probs), key=lambda x: x[1], reverse=True)[:top_n]
    return [(disease, round(prob * 100, 2)) for disease, prob in top_preds]

In [27]:
# Test loading and prediction
sample_input = "sweating, low blood pressure, vomiting"
predictions = load_and_predict('symptom_classifier_ensemble.pkl', sample_input)
print("\nTop Predicted Diseases:")
for i, (disease, prob) in enumerate(predictions, 1):
    print(f"{i}. {disease}: {prob}%")


Top Predicted Diseases:
1. urinary tract infection: 26.01%
2. peptic ulcer disease: 6.21%
3. Dengue: 5.91%
